<a href="https://colab.research.google.com/github/yuezhang23/nlp/blob/main/reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Sentiment Labels

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents.  As we discussed in class, many classification models represent a document as a sparse vector of of features and their weights. Weights could be boolean, counts, tf-idf values, or other functions of the document. In a linear model, we then combine the document vectors with the candidate output classes and learn weights for each of the feature-class pairs. This last step is implemented for you.

For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

There are many helpful resources online for getting up to speed with vector representations of documents. One example is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [9]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np

In [10]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [32]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import lil_matrix, csr_matrix
import re
# pprint.pprint(texts[1])
def createBasicFeatures(corpus):
  # Your code here
  # create a list of tokens for each document
  tokens = []
  vocab = set()
  for doc in corpus:
    text = doc['text']
    # add any alphbetic word with lowercase
    tks = re.split(r'\W+', text)
    tks = [tk.lower() for tk in tks if tk and tk.isalpha()]
    tokens.append(tks)
    # collect unique tokens
    vocab.update(set(tks))

  vocab = list(vocab)
  vocab_idx = {v: i for i, v in enumerate(vocab)}
  # create a sparse matrix
  texts = lil_matrix((len(corpus), len(vocab)), dtype=np.float64)
  for i in range(len(tokens)):
    tks = tokens[i]
    for tk in tks:
      texts[i, vocab_idx[tk]] += 1
  texts = texts.tocsr()

  # compare to count vectorizer
  # vectorizer = CountVectorizer(token_pattern=r'\b[a-zA-Z]+\b', lowercase=True, min_df=1)
  # texts = [txt['text'] for txt in corpus]
  # texts = vectorizer.fit_transform(texts)
  # vocab = vectorizer.get_feature_names_out().tolist()

  classes = [txt['class'].lower() for txt in corpus]
  return texts,classes,vocab

In [36]:
# TODO: Implement createFancyFeatures andn describe in comments what the
# features are and why they might ben helpful.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
  #Your code here
  # using tfidf score for feature representation
  vectorizer = TfidfVectorizer(
    token_pattern=r'\b[a-zA-Z]+\b',
    stop_words='english',
    ngram_range=(1, 1),
    lowercase=True,
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True
  )

  print("n_gram range : {1, 1}")
  texts = [text['text'] for text in corpus]
  classes = [text['class'].lower() for text in corpus]

  texts = vectorizer.fit_transform(texts)
  # get feature array in same ordering
  vocab = vectorizer.get_feature_names_out().tolist()

  return texts,classes,vocab

In [13]:
# given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  # create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))

  # determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  # determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [14]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [30]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.826500
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'edge', 'sherri', 'excellent', 'perfectly', 'masterpiece', 'enjoyable', 'fun', 'using', 'overall', 'gas', 'solid', 'quite', 'command', 'different', 'fantastic', 'liar', 'follows']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'lame', 'worst', 'headed', 'awful', 'unfortunately', 'cheap', 'write', 'boring', 'superior', 'tedious', 'bad', 'jesse', 'terrible', 'poor', 'maybe', 'looks', 'jakob']
----------L2 Norm-----------
The model's average accuracy is 0.832000
The most informative terms for pos are: ['fun', 'back', 'great', 'quite', 'well', 'excellent', 'overall', 'seen', 'american', 'perfectly', 'yet', 'memorable', 'terrific', 'pulp', 'job', 'very', 'true', 'performances', 'different', 'solid']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'boring', 'only', 'script', 'awf

Run the following to train and evaluate two models using extended features:

In [35]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

n_gram range : {1, 1}
----------L1 Norm-----------
The model's average accuracy is 0.750000
The most informative terms for pos are: ['great', 'life', 'perfectly', 'perfect', 'quite', 'seen', 'memorable', 'excellent', 'different', 'hilarious', 'performances', 'overall', 'performance', 'true', 'terrific', 'world', 'war', 'best', 'especially', 'jackie']
The most informative terms for neg are: ['bad', 'worst', 'boring', 'supposed', 'waste', 'ridiculous', 'mess', 'awful', 'stupid', 'wasted', 'plot', 'script', 'reason', 'dull', 'unfortunately', 't', 'lame', 'looks', 'attempt', 'poor']
----------L2 Norm-----------
The model's average accuracy is 0.857000
The most informative terms for pos are: ['great', 'life', 'perfect', 'hilarious', 'excellent', 'perfectly', 'memorable', 'terrific', 'quite', 'true', 'best', 'different', 'world', 'performances', 'performance', 'fun', 'especially', 'overall', 'wonderful', 'family']
The most informative terms for neg are: ['bad', 'worst', 'boring', 'plot', 'su

**TODO**: Briefly comment on your results. You do _not_ need to beat the basic features model to get full credit.

| Feature Type | N-gram range | L1 Accuracy | L2 Accuracy |
|:------------|:-------|------------:|------------:|
| TF-IDF | (1, 1) | 75.00% | **85.70%** |
| TF-IDF | (1, 2) | 67.15% | 83.60% |
| TF-IDF | (2, 2) | 48.30% | 77.35% |
| Raw Frequencies | (1, 1) | **82.65%** | 83.20% |

Based on the Table,

1.   BaseFeature (Raw Frequencies) implementation has the highest accuracy in L1 regularization while Fancyfeature (TF-IDF) implementation has the highest accuracy in L2 regularization.
2.   Fancyfeature shows much more improvement with L2 vs L1 (10.7% increase from 0.75 to 0.857), while BaseFeature performs more consistently across both regularizations. This occurs because L1 on TF-IDF creates a 'double penalty' effect by removing rare words that may have rich sentiment.BaseFeature avoids this as common sentiment words maintain high counts
that survive L1's feature selection.
3.   In this case unigram models have the best performance, indicating that the given corpus has direct sentiment expressions rather than complex contextual patterns.


